In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: ContinuousCallbackPMS - ContinuousPMS Sonecule using a Callback function for flexible mapping

This notebook introduces and demonstrates usage of the `ContinuousCallbackPMS` sonecule.
* The sonecule spawns a synth to be modulated by a data-mapped parameters
* The synths can be defined as wished with parameters as needed
* The default synth offers the following parameters:
  * amplitude
  * frequency
  * sharpness
  * spatial panning
* Different from the `ContinuousPMS`, where the mapping is specified by a mapping specification dictionary that is parsed column by column, this sonecule calls a callback function for each row, providing the row vector as an argument. 
* this allows highly flexible mappings and even things that are impossible with the non-callback Sonecule, such as playing a sound that conveys which channel has the highest value.
* Definition of a callback function is a daunting task for non-programmers. Therefore the sonecule comes with a create_callback_template method, that delivers a function that can be copied and pasted in a jupyter notebook cell (or your IDE), as starting point for own sonification designs.


## Data Preparation

In [ ]:
# headers and imports for the demo
import sonecules as sn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx

In [ ]:
%run ../data/prepare-data.ipynb

For this sonecule we work with the building data set for the demonstrations. That is a data set that describes the hourly water and electricity consumption of an office building plus environmental features such as temperature, humidity, wind-speed, time of day, solar radiation. With roughly 4300 rows the data set contains roughly half a year of data.

In [ ]:
df = dataframes["building"]
df.plot(lw=0.2);

## Usage / Demo Snippets

Let's create a ContinuousCallbackPMS using a callback function for mapping.

First we need to import the sonecule

In [ ]:
from sonecules.scoreson import ContinuousCallbackPMS, mapcol

The following cell shows all steps usually used to sonify the data, i.e., 
- creation of the sonecule
- cleaning the time line
- defining the callback function (here named callback_fn, but any name is fine)
- creating the sonification using the sonecules schedule function
- starting your sonification using the sonecule start function

For the example we start with one week =7*24 hourly measurments of the building data set

Note that the callback function receives the following arguments:
- r: the row vector from the data set
- cmi: the dictionary of all columns' minimum values (useful for the mapcol function)
- cma: the dictionary of all columns' maximum values (useful for the mapcol function)
- pp: the dictionary containing all parameters as keys and their default values as value
  - so if pp entries are not overwritten, the default will be used

For mapping any python based code can be used, including if/else statements and computations.
- a common case is to map values of a specific feature (e.g. column $k$) to the parameter
- to simplify this usecase, we offer the `mapcol(r, column_name, cmi, cma, p1, p2)` function:
  - it maps column `column_name` (string or integer)
  - from source range [ cmi[column_name], cma[column_name] ] 
  - to a target range [p1, p2], note that p1 < p2 is not necessary...
  - using a linear mapping function without clipping.
- Any nonlinear warping can happen as developers see fit, e.g. see mapping for 'amp' below

- **Please note that at the moment data are assumed to be sorted by time and row index is linearly mapped from 0 to duration. This behavior will probably change in future revisions of the sonecule towards explicit mapping on the virtual 'onset' parameter, same as it is already implemented in DiscreteCallbackPMS**.

In [ ]:
# create the sonecule
scb = ContinuousCallbackPMS(df.iloc[:7*24, 6:])  # one week, omit the first 6 columns

# reset the timeline
ctx.timeline.reset()

# we use the mapcol helper function which maps the columns data 
# mapcol(data_row, feature, column_mins, column_maxs, target_min, target_max)

# define the callback function as needed/wanted
def callback_fn(r, cmi, cma, pp):
    pp['freq']      = pam.midi_to_cps(mapcol(r, 'temperature', cmi, cma, 48, 72))
    pp['amp']       = pam.db_to_amp(mapcol(r, 'humidity', cmi, cma, -20, 0)) 
    pp['pan']       = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    pp['numharm']   = mapcol(r, 'solar_radiation', cmi, cma, 1, 12)
    pp['vibfreq']   = pam.linlin(r['hc_wb_hot_water'], -0.5, 0.5, 3, 8)
    pp['vibintrel'] = 0
    return pp # it returns the mapping dictionary

# sonify the data using the above callback function
scb.schedule(at=0, duration=6, callback_fn=callback_fn)

# start the playback 
scb.start(rate=1)

- play with the definition of the callback_fn and execute again to explore the data
- control the duration and data slice.
- Once you want to keep the callback function fixed there is no need to reexecute
- Here some other data sonified with the same callback function

In [ ]:
ctx.reset()
scb = ContinuousCallbackPMS(df.iloc[30*24:60*24, 7:])  # the second month of data in 10 seconds
scb.schedule(at=0, duration=10, callback_fn=callback_fn).start()

In [ ]:
ctx.reset()
scb = ContinuousCallbackPMS(df.iloc[::, 7:])  # all data, every row in 3.5 seconds
scb.schedule(at=0, duration=3.5, callback_fn=callback_fn).start()

- As this is all executed in real-time, you will probably experience computation limits of your system.
- Sonecules will issue late warnings if there are processing-based delays
- in such cases, a non-real-time rendering is always an option to compute a guaranteed correct sonification. Good that it is easy to render the mesonic Context

In [ ]:
asig = ctx.render_asig()  # or ctx.render("filename.wav")
asig

In [ ]:
# plot first two audio channels
plt.figure()
asig[:,:2].plot(offset=1, lw=0.5)

In [ ]:
# to play the asig, we'd have to start the pya.Aserver
from pya import startup
aserver = startup()

In [ ]:
# now we can play, also with different rates (speeds)
asig.play(rate=1, server=aserver)

Note that the duration of the Asig matches `timeline.end_time`

In [ ]:
ctx.timeline.end_time

...which can be set manually or automatically by using `timeline.end_time_offset`

In [ ]:
print(f"The last TimeBundle in the Timeline is at {ctx.timeline.last_timestamp}")
print(f"The offset is {ctx.timeline.end_time_offset}")
print(f"Therefore timeline.end_time = {ctx.timeline.last_timestamp + ctx.timeline.end_time_offset}")

Let's look at our rendering as asig

Now let the sonecule propose a mapping as starting point for own experiments.

- Calling `create_callback_template()` composes 
- and then prints the python code string.
- This output can be copied into a notebook cell and adapted as needed.
- On execution it defines the callback function cbfn().
- The sonification can be rendered as you go until satisfactory.

In [ ]:
scb = ContinuousCallbackPMS(df.iloc[:7*24, 8:])  # first week but no weekday features
fnstr = scb.create_callback_template(auto_assign=True)

The printout shows python code 
- that you can simply copy and paste into a new code cell
- modify the cell as you see fit, e.g. remove rows, or set the right hand side to a constant as preferred
- once satisfied, you can execute the function.
- Note that the 


In [ ]:
def cbfn(r, cmi, cma, pp):
    # columns are: 'hour' 'am_pm' 'temperature' 'humidity' 
    # 'solar_radiation' 'wind_speed' 'hc_wb_electrical' 'hc_wb_cold_water' 
    # 'hc_wb_hot_water' 
    pp['freq']     	 = mapcol(r, 'hour', cmi, cma, 300.00, 600.00)
    pp['amp']      	 = mapcol(r, 'am_pm', cmi, cma, 0.08, 0.15)
    pp['vibfreq']  	 = mapcol(r, 'temperature', cmi, cma, 0.00, 0.00)
    pp['vibintrel']	 = mapcol(r, 'humidity', cmi, cma, 0.00, 0.00)
    pp['numharm']  	 = mapcol(r, 'solar_radiation', cmi, cma, 0.00, 10.00)
    pp['pulserate']	 = mapcol(r, 'wind_speed', cmi, cma, 0.00, 0.00)
    pp['pint']     	 = mapcol(r, 'hc_wb_electrical', cmi, cma, 0.00, 0.00)
    pp['pwid']     	 = mapcol(r, 'hc_wb_cold_water', cmi, cma, 0.75, 1.50)
    pp['pan']      	 = mapcol(r, 'hour', cmi, cma, -1.00, 1.00)
    return pp
# create sonification e.g. by using
sn.gcc().timeline.reset()
scb.schedule(at=0, duration=5, callback_fn=cbfn).start(rate=1)

And finally here is a hand-crafted mapping, from modifying the code

In [ ]:
scb = ContinuousCallbackPMS(df.iloc[14*24:18*24, 8:]) 

def cbfn(r, cmi, cma, pp):
    # columns are:
    # 'hour' 'am_pm' 'temperature' 'humidity' 
    # 'solar_radiation' 'wind_speed' 'hc_wb_electrical' 'hc_wb_cold_water' 
    # 'hc_wb_hot_water' 
    # print(r, pp)
    pp['freq']	     = mapcol(r, 'solar_radiation', cmi, cma, 100, 400)
    pp['amp']	     = mapcol(r, 'humidity', cmi, cma, 0, 1)
    pp['numharm']	 = mapcol(r, 'am_pm', cmi, cma, 1, 4)
    pp['vibintrel']  = 0
    pp['pan']	     = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    return pp

ctx.timeline.reset()
scb.schedule(at=1, duration=8, callback_fn=cbfn).start(rate=1)

In [ ]:
ctx.stop()

In [ ]:
# close context only if it is not needed anymore...
# ctx.close()

## Code Template

The following code snippets are intended for copy & paste to your notebooks, to facilitate getting your data sonified
using this sonecule.

In [ ]:
from sonecules.scoreson import ContinuousCallbackPMS, mapcol

In [ ]:
# load your multi-channel data into an Asig, e.g. 
data = np.random.random((400, 4))-0.5 # 100 rows with 8 channels, here same fake data
data = np.cumsum(data,axis=0)
df = pd.DataFrame(data, columns=["c1", "c2", "c3", "c4"])
df.plot(subplots=True);
df.head()

Next (optionally) we could define our synth.
The example shows it for the sc3nb backend, for the pya backend this will be simply an Asig generating python function

In [ ]:
ctx.synths.add_synth_def("csharpsyn",
    r"""{ | out=0, freq=400, amp=0.2, vibfreq=0, vibir=0, sharp=0, pan=0 |
    var vib = SinOsc.ar(vibfreq, mul: vibir*freq, add: freq);
    var sig = HPF.ar(Formant.ar(vib, vib, bwfreq: vib*(sharp+1), mul: amp), 40);
    Out.ar(out, Pan2.ar(sig, pan, amp));
}""")

In [ ]:
# load your data / select your data
mydf = df

# sonecule for your synth with defaults and bounds
scb = ContinuousCallbackPMS(mydf, "csharpsyn")

In [ ]:
scb.create_callback_template(auto_assign=True)

Now copy the output to a new cell (below)
- uncomment the last line, make sure the sonecule variable name match that before .schedule
- modify your callback function as needed

In [ ]:
def cbfn(r, cmi, cma, pp):
    # columns are: 'c1' 'c2' 'c3' 'c4' 
    pp['freq']     	 = mapcol(r, 'c1', cmi, cma, 300.00, 600.00)
    pp['amp']      	 = mapcol(r, 'c2', cmi, cma, 0.15, 0.90)
    pp['vibfreq']  	 = mapcol(r, 'c3', cmi, cma, 20.00, 50.00)
    pp['vibir']    	 = mapcol(r, 'c4', cmi, cma, 0.00, 0.10)
    pp['sharp']    	 = mapcol(r, 'c1', cmi, cma, 0.00, 5.00)
    pp['pan']      	 = mapcol(r, 'c2', cmi, cma, -1.00, 1.00)
    return pp
# create sonification e.g. by using
sn.gcc().timeline.reset()
scb.schedule(at=0, duration=5, callback_fn=cbfn).start(rate=1)

In [ ]:
ctx.stop()

In [ ]:
ctx.close()